<a href="https://colab.research.google.com/github/fjme95/algebra-lineal-para-machine-learning/blob/main/7_Textrank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Implementación de TextRank para la obtención de resúmenes

En este Notebook se implementará TextRank para obtener un resumen con las oraciones clave de todo un texto.

# Dependencias

In [1]:
%%capture
!pip install wikipedia git+https://github.com/neuml/txtai#egg=txtai[pipeline]

In [2]:
import re

import pandas as pd
import numpy as np
import scipy.linalg as splinalg

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

import wikipedia

from txtai.pipeline import Translation



In [3]:
nltk.download("punkt")
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# Radicalizador
stemmer = PorterStemmer()

# Palabras de paro
cached_stopwords = stopwords.words('english')
cached_stopwords[:10]

# Traductor
translate = Translation()

# Datos

Los datos que ocuparemos serán el texto de páginas de Wikipedia. Descargaremos el texto ocupando el módulo [```wikipedia```](https://pypi.org/project/wikipedia/) que es un "wrapper" del API de Wikipedia. A este texto lo dividiremos en oraciones, procesaremos cada oración, radicalizaremos cada palabra, y aplicaremos TextRank para obtener las oraciones más importantes de todo el documento.

## Lectura de los datos

Descargamos un artículos de Wikipedia.

In [5]:
wiki = wikipedia.page('Expropiación del petróleo en México')
book = wiki.content
print(book)

The Mexican oil expropriation (Spanish: expropiación petrolera) was the nationalization of all petroleum reserves, facilities, and foreign oil companies in Mexico on March 18, 1938.  In accordance with Article 27 of the Constitution of 1917, President Lázaro Cárdenas declared that all mineral and oil reserves found within Mexico belong to "the nation", i.e., the federal government. The Mexican government established a state-owned petroleum company, Petróleos Mexicanos, or PEMEX.  For a short period, this measure caused an international boycott of Mexican products in the following years, especially by the United States, the United Kingdom, and the Netherlands, but with the outbreak of World War II and the alliance between Mexico and the Allies, the disputes with private companies over compensation were resolved. The anniversary, March 18, is now a Mexican civic holiday.


== Background ==

On August 16, 1935, the Petroleum Workers Union of Mexico (Sindicato de Trabajadores Petroleros de

## Procesamiento

Dividimos el texto en oraciones.

In [6]:
sentences = [x for x in sent_tokenize(book)]
print(f"# oraciones: {len(sentences)}")
for sentence in sentences[:3]:
    print(sentence)
    print()
    print("...Fin de la oración...")
    print()


# oraciones: 94
The Mexican oil expropriation (Spanish: expropiación petrolera) was the nationalization of all petroleum reserves, facilities, and foreign oil companies in Mexico on March 18, 1938.

...Fin de la oración...

In accordance with Article 27 of the Constitution of 1917, President Lázaro Cárdenas declared that all mineral and oil reserves found within Mexico belong to "the nation", i.e., the federal government.

...Fin de la oración...

The Mexican government established a state-owned petroleum company, Petróleos Mexicanos, or PEMEX.

...Fin de la oración...



convertimos a minúsculas, eliminamos stopwords, eliminamos signos de puntuación y radicalizamos.

In [7]:
sent_low = [[stemmer.stem(re.sub('[^a-z]', "", word.lower())) for word in word_tokenize(sentence) if word not in cached_stopwords and len(word) > 2] for sentence in sentences]
sent_low[3]

['for',
 'short',
 'period',
 'measur',
 'caus',
 'intern',
 'boycott',
 'mexican',
 'product',
 'follow',
 'year',
 'especi',
 'unit',
 'state',
 'unit',
 'kingdom',
 'netherland',
 'outbreak',
 'world',
 'war',
 'allianc',
 'mexico',
 'alli',
 'disput',
 'privat',
 'compani',
 'compens',
 'resolv']

# TextRank

Construimos la matriz de adyacencias/similitud A entre las oraciones, tomando el número de palabras que están en ambas como la similitud entre las dos oraciones.

In [8]:
A = np.zeros((len(sent_low), len(sent_low)))

for i in range(len(sentences)):
    if i % 100 == 0:
        print(i, end=", ")
        if i % 1000 == 0:
            print()
    for j in range(i+1, len(sentences)):
        # La simillitud entre oraciones va a ser el número de palabras que tienen en común
        A[i][j] = A[j][i] = len([x for x in sentences[i] if x in sentences[j]])

0, 


Así es como se ve un fragmento de la matriz A.

In [9]:
A[:5, :5]

array([[  0., 164., 167., 169., 151.],
       [164.,   0., 181., 186., 160.],
       [167., 181.,   0.,  90.,  77.],
       [169., 186.,  90.,   0., 270.],
       [151., 160.,  77., 270.,   0.]])

Normalizamos las columnas de A

In [10]:
suma = np.sum(A, axis=0)
A_norm = np.divide(A, suma, where=suma!=0)
A_norm[:5, :5]

array([[0.        , 0.01004348, 0.02120366, 0.0061283 , 0.02966601],
       [0.01122058, 0.        , 0.02298121, 0.00674475, 0.03143418],
       [0.01142583, 0.01108457, 0.        , 0.00326359, 0.0151277 ],
       [0.01156267, 0.01139078, 0.01142712, 0.        , 0.05304519],
       [0.01033114, 0.00979852, 0.00977654, 0.00979077, 0.        ]])

Se crea el vector de TextRank con unos y se itera hasta que converja. Es decir, hasta que obtengamos $\Pi$ tal que $$\Pi = A~\Pi$$ 

In [11]:
tol = 1e-7

PI_ = np.ones(A_norm.shape[1])
    
i = 0
while True:
    pi_ = A_norm.dot(PI_)
    print(i, abs(PI_- pi_).sum()) 
    if np.allclose(PI_, pi_, tol):
        break
    i += 1
    PI_ = pi_

0 25.119042342061313
1 3.9834907908161314
2 0.674362222598042
3 0.11271087281569214
4 0.019067876555544372
5 0.0032164628903874615
6 0.0005432400722329978
7 9.173100230158715e-05
8 1.549020578212046e-05
9 2.6160075131675953e-06


Alternativamente, podemos obtener los eigenvectores izquierdos de nuestra matriz A_norm. Los valores de PageRank corresponden al vector de probabilidades del estado estacionario de la matriz A que a su vez es el eigenvector izquierdo con eigenvalor asociado 1.

$$\Pi = \Pi A^T$$

In [12]:
_, vecs = splinalg.eig(A_norm.T, left=True, right=False)

In [13]:
pi_ = vecs[:, 0]

Obtenemos los índices de los k valores más grandes en $\Pi$ y los usamos para obtener las oraciones más relevantes.

In [14]:
k = 3
pi_.argsort()[-k:][::-1]

array([5, 3, 6])

In [15]:
summary = [sentences[idx] for idx in pi_.argsort()[-k:][::-1]]

Por último, sólo queda ver qué considero TextRank como las oraciones más importantes.

In [16]:
for bullet in summary:
    print()
    print(bullet)


== Background ==

On August 16, 1935, the Petroleum Workers Union of Mexico (Sindicato de Trabajadores Petroleros de la República Mexicana) was formed and one of the first actions was the writing of a lengthy draft contract transmitted to the petroleum companies demanding a 40-hour working week, a full salary paid in the event of sickness, and the payment of 65 million pesos towards benefits and wages.

For a short period, this measure caused an international boycott of Mexican products in the following years, especially by the United States, the United Kingdom, and the Netherlands, but with the outbreak of World War II and the alliance between Mexico and the Allies, the disputes with private companies over compensation were resolved.

The foreign oil companies refused to sign the agreement, and counter offered with a payment of 14 million pesos toward wages and benefits.On November 3, 1937, the union demanded that the companies sign the collective agreement and on May 17, the union s

Podemos traducir la salida.

In [17]:
for bullet in summary:
    print()
    print(translate(bullet, "es"))


== Antecedentes ==El 16 de agosto de 1935, se formó el Sindicato de Trabajadores Petroleros de la República Mexicana y una de las primeras acciones fue la redacción de un largo borrador de contrato transmitido a las compañías petroleras exigiendo una semana laboral de 40 horas, un salario completo pagado en caso de enfermedad y el pago de 65 millones de pesos por beneficios y salarios.

Durante un corto período, esta medida provocó un boicot internacional a los productos mexicanos en los años siguientes, especialmente por parte de los Estados Unidos, el Reino Unido y los Países Bajos, pero con el estallido de la Segunda Guerra Mundial y la alianza entre México y los Aliados, se resolvieron las disputas con empresas privadas por la compensación.

El 3 de noviembre de 1937, el sindicato exigió a las empresas que firmaran el convenio colectivo y el 17 de mayo, el sindicato convocó una huelga en caso de que no se cumplieran sus demandas.


# Función para crear resúmenes

Podemos condensar todo lo anterior en una función que reciba texto y nos regrese las oraciones más relevantes de acuerdo a TextRank.

In [39]:
def summary(text, k, to_spanish = True, tol = 1e-5, d = .15, eig = False):
    print("Paso 1. Obteniendo oraciones")
    sentences = [x for x in sent_tokenize(text)]

    print(f"# oraciones: {len(sentences)}")
    
    print("Paso 2. Procesando texto")
    sent_low = [[stemmer.stem(re.sub('[^a-z]', "", word.lower())) for word in word_tokenize(sentence) if word not in cached_stopwords and len(word) > 2] for sentence in sentences]
    
    print("Paso 3. Creando matriz de similitud")
    A = np.zeros((len(sent_low), len(sent_low)))
    
    for i in range(len(sentences)):
        for j in range(i+1, len(sentences)):
            # La simillitud entre oraciones va a ser el número de palabras que tienen en común
            A[i][j] = A[j][i] = len([x for x in sentences[i] if x in sentences[j]])

    print("Paso 4. Normalizando matriz de similitud")   
    suma = np.sum(A, axis=0)
    A_norm = np.divide(A, suma, where=suma!=0)
    
    print("Paso 5. Ejecutando TextRank")
    if eig:
        vals, vecs = splinalg.eig(A_norm.T, left=True, right=False)
        pi_ = vecs[:, 0]
    else:
        PI_ = np.ones(A_norm.shape[1])
        
        while True:
            pi_ = A_norm.dot(PI_)
            if np.allclose(PI_, pi_, tol):
                break
            PI_ = pi_

    print("\tPaso 5. Terminado")

    if not to_spanish:
        return [sentences[idx] for idx in pi_.argsort()[-k:][::-1]]

    print("Paso 6. Traduciendo")
    return [translate(sentences[idx], "es") for idx in pi_.argsort()[-k:][::-1]]

def print_bullet_points(bullet_points):
    for point in bullet_points:
        print(f"- {point}\n")


In [47]:
wiki = wikipedia.page('Automatic summarization')
text = wiki.content
bullet_points = summary(text, 5, False, eig = False)

Paso 1. Obteniendo oraciones
# oraciones: 312
Paso 2. Procesando texto
Paso 3. Creando matriz de similitud
Paso 4. Normalizando matriz de similitud
Paso 5. Ejecutando TextRank
	Paso 5. Terminado


In [48]:
print_bullet_points(bullet_points)

- Consider the example text from a news article:

"The Army Corps of Engineers, rushing to meet President Bush's promise to protect New Orleans by the start of the 2006 hurricane season, installed defective flood-control pumps last year despite warnings from its own expert that the equipment would fail during a storm, according to documents obtained by The Associated Press".A keyphrase extractor might select "Army Corps of Engineers", "President Bush", "New Orleans", and "defective flood-control pumps" as keyphrases.

- Text summarization finds the most informative sentences in a document; various methods of image summarization are the subject of ongoing research, with some looking to display the most representative images from a given collection or generating a video; video summarization extracts the most important frames from the video content.

- For text, extraction is analogous to the process of skimming, where the summary (if available), headings and subheadings, figures, the fir

In [51]:
!wget https://www.gutenberg.org/files/84/84-0.txt -O book.txt

--2021-10-15 08:41:14--  https://www.gutenberg.org/files/84/84-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 448821 (438K) [text/plain]
Saving to: ‘book.txt’

book.txt            100%[===================>] 438.30K  1.17MB/s    in 0.4s    

2021-10-15 08:41:15 (1.17 MB/s) - ‘book.txt’ saved [448821/448821]



In [54]:
with open("book.txt") as f:
    book_raw = f.read()
print(book_raw[0:1000])

﻿The Project Gutenberg eBook of Frankenstein, by Mary Wollstonecraft (Godwin) Shelley

This eBook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this eBook or online at
www.gutenberg.org. If you are not located in the United States, you
will have to check the laws of the country where you are located before
using this eBook.

Title: Frankenstein
       or, The Modern Prometheus

Author: Mary Wollstonecraft (Godwin) Shelley

Release Date: 31, 1993 [eBook #84]
[Most recently updated: November 13, 2020]

Language: English

Character set encoding: UTF-8

Produced by: Judith Boss, Christy Phillips, Lynn Hanninen, and David Meltzer. HTML version by Al Haines.
Further corrections by Menno de Leeuw.

*** START OF THE PROJECT GUTENBERG EBOOK FRANKENSTEIN ***




Frankenstein;

or, the Modern Pro

In [ ]:
bullet_points = summary(book_raw, 5, False, eig = True)

Paso 1. Obteniendo oraciones
# oraciones: 3196
Paso 2. Procesando texto
Paso 3. Creando matriz de similitud


In [ ]:
print_bullet_points(bullet_points)

# Ejercicios

## Matriz de similitud entre oraciones

Para la similitud entre las oraciones se uso el número de palabras que aparecen en ambas. **Reemplazar por similitud coseno** y comparar los resultados.

Un muy buen primer acercamiento podría ser ocupando Latent Semantic Analysis y calcular la similitud coseno entre todos los documentos.

Si tienen una DataFrame con las columnas ```[id_documento_1, id_documento_2, similitud]```, usar la función [```pandas.DataFrame.unstack```](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.unstack.html) puede ayudar a crear la matriz de similitud.




## Idioma

Este ejemplo esta hecho para texto en inglés por las stopwords que se usan y el radicalizador (PorterStemmer). Hacer los cambios necesarios para que reciba textos en español.

Esto es, cambiar las stopwords (nltk tiene stopwords en español) y el radicalizador (Pista: ```nltk.stemmer``` tiene más radicalizadores y uno de ellos tienen un algoritmo para el español)

## Oraciones vs. Palabras

En este Notebook ocupamos las oraciones para obtener el resumen, de haber ocupado las palabras, de TextRank obtendríamos las palabras clave del texto. 

Implementar TextRank con palabras. Para la matriz de similitud (o adyacencias), se pueden ligar las palabras que son consecutivas o definir una ventana de k palabras consecutivas en cada oración (parecido a skip-gram) y ligar todas estas palabras. En este caso, la matriz A tendría la dimensión del vocabulario (lista de palabras únicas) y tendría un 1 si las palabras están ligadas.

Una alternativa más sería ocupar un embedding de palabras (e.g. word2vec) y calcular la similitud coseno entre los vectores de cada palabra para llenas a A.

Después de eso, todo sería o mismo.

## Resumen sobre un tema

Aquí ocupamos sólo un documento para aplicarle TextRank. Podemos tener un corpus de documentos del mismo tema (e.g. noticias del caso de los 31 científicos) y aplicarlo para obtener los puntos importantes de todo el corpus.

A la implementación actual no se le tiene que cambiar nada, sólo concatenar en una sola cadena de texto todo el corpus.

Ejercicio: Construir un copus con 4 artículos sobre un tema de interés, concatenarlos y pasarlo como parámetro a la función ```summary```.

## Mejorar la función ```summary```

Podemos dividir el código de la función para que funcionen como módulos y permita cierta libertad a la hora de ejecutarse. Por ejemplo, podríamos tener varias funciones que calculen la matriz A de diferentes maneras y que dentro de ```summary``` se ejecute una de tantas de acuerdo a un parámetro de la función.

Ejercicio: Crear funciones para cada paso de ```summary```

# Sobre la obtención de los valores de PageRank

https://nlp.stanford.edu/IR-book/html/htmledition/the-pagerank-computation-1.html

https://nlp.stanford.edu/IR-book/html/htmledition/markov-chains-1.html